In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  #use 2080Ti


In [2]:
from keras.applications import VGG16

#conv_base = VGG16(weights='imagenet',
conv_base = VGG16(weights=None,
                  include_top=False,
                  input_shape=(224, 224, 3))

Using TensorFlow backend.
W0812 22:37:45.003679 139723895002944 deprecation_wrapper.py:119] From /home/chtseng/envs/AI/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 22:37:45.014776 139723895002944 deprecation_wrapper.py:119] From /home/chtseng/envs/AI/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 22:37:45.016719 139723895002944 deprecation_wrapper.py:119] From /home/chtseng/envs/AI/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 22:37:45.035562 139723895002944 deprecation_wrapper.py:119] From /home/chtseng/envs/AI/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_po

In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = '/home/chtseng/works/report/dataset-diabnext'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)

def extract_features(directory, sample_count, batch_size):
    features = np.zeros(shape=(sample_count, 7, 7, 512))
    labels = np.zeros(shape=(sample_count, 10))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch

        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 114, batch_size = 6)
validation_features, validation_labels = extract_features(validation_dir, 37, batch_size = 37)
test_features, test_labels = extract_features(test_dir, 37, batch_size = 37)

Found 1140 images belonging to 10 classes.


W0812 22:37:49.127231 139723895002944 deprecation_wrapper.py:119] From /home/chtseng/envs/AI/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0812 22:37:49.129205 139723895002944 deprecation_wrapper.py:119] From /home/chtseng/envs/AI/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

/home/chtseng/envs/AI/lib/python3.6/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Found 370 images belonging to 10 classes.
Found 370 images belonging to 10 classes.


In [5]:
train_features = np.reshape(train_features, (114, 7 * 7 * 512))
validation_features = np.reshape(validation_features, (37, 7 * 7 * 512))
test_features = np.reshape(test_features, (37, 7 * 7 * 512))

In [6]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=7 * 7 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=200,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

W0812 22:38:03.656832 139723895002944 deprecation.py:506] From /home/chtseng/envs/AI/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0812 22:38:03.684683 139723895002944 deprecation_wrapper.py:119] From /home/chtseng/envs/AI/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0812 22:38:03.808791 139723895002944 deprecation.py:323] From /home/chtseng/envs/AI/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule a

Train on 114 samples, validate on 37 samples
Epoch 1/200
114/114 [==============================] - 1s 6ms/step - loss: 2.3028 - acc: 0.0877 - val_loss: 2.3024 - val_acc: 0.1351
Epoch 2/200
114/114 [==============================] - 0s 505us/step - loss: 2.3024 - acc: 0.1491 - val_loss: 2.3025 - val_acc: 0.1351
Epoch 3/200
114/114 [==============================] - 0s 528us/step - loss: 2.3021 - acc: 0.1579 - val_loss: 2.3025 - val_acc: 0.1351
Epoch 4/200
114/114 [==============================] - 0s 470us/step - loss: 2.3020 - acc: 0.1579 - val_loss: 2.3025 - val_acc: 0.1351
Epoch 5/200
114/114 [==============================] - 0s 510us/step - loss: 2.3019 - acc: 0.1491 - val_loss: 2.3025 - val_acc: 0.1351
Epoch 6/200
114/114 [==============================] - 0s 492us/step - loss: 2.3018 - acc: 0.1491 - val_loss: 2.3026 - val_acc: 0.1351
Epoch 7/200
114/114 [==============================] - 0s 477us/step - loss: 2.3019 - acc: 0.1579 - val_loss: 2.3026 - val_acc: 0.1351
Epoch 8/200


Epoch 61/200
114/114 [==============================] - 0s 482us/step - loss: 2.2943 - acc: 0.1491 - val_loss: 2.3027 - val_acc: 0.1351
Epoch 62/200
114/114 [==============================] - 0s 464us/step - loss: 2.2945 - acc: 0.1491 - val_loss: 2.3027 - val_acc: 0.1351
Epoch 63/200
114/114 [==============================] - 0s 471us/step - loss: 2.2952 - acc: 0.1491 - val_loss: 2.3028 - val_acc: 0.1351
Epoch 64/200
114/114 [==============================] - 0s 480us/step - loss: 2.2963 - acc: 0.1491 - val_loss: 2.3028 - val_acc: 0.1351
Epoch 65/200
114/114 [==============================] - 0s 506us/step - loss: 2.2948 - acc: 0.1491 - val_loss: 2.3028 - val_acc: 0.1351
Epoch 66/200
114/114 [==============================] - 0s 486us/step - loss: 2.2958 - acc: 0.1491 - val_loss: 2.3028 - val_acc: 0.1351
Epoch 67/200
114/114 [==============================] - 0s 507us/step - loss: 2.2969 - acc: 0.1491 - val_loss: 2.3028 - val_acc: 0.1351
Epoch 68/200
114/114 [==========================

114/114 [==============================] - 0s 472us/step - loss: 2.2898 - acc: 0.1491 - val_loss: 2.3035 - val_acc: 0.1351
Epoch 122/200
114/114 [==============================] - 0s 491us/step - loss: 2.2890 - acc: 0.1491 - val_loss: 2.3035 - val_acc: 0.1351
Epoch 123/200
114/114 [==============================] - 0s 478us/step - loss: 2.2893 - acc: 0.1491 - val_loss: 2.3035 - val_acc: 0.1351
Epoch 124/200
114/114 [==============================] - 0s 481us/step - loss: 2.2910 - acc: 0.1491 - val_loss: 2.3035 - val_acc: 0.1351
Epoch 125/200
114/114 [==============================] - 0s 496us/step - loss: 2.2870 - acc: 0.1491 - val_loss: 2.3035 - val_acc: 0.1351
Epoch 126/200
114/114 [==============================] - 0s 476us/step - loss: 2.2918 - acc: 0.1491 - val_loss: 2.3036 - val_acc: 0.1351
Epoch 127/200
114/114 [==============================] - 0s 503us/step - loss: 2.2908 - acc: 0.1491 - val_loss: 2.3036 - val_acc: 0.1351
Epoch 128/200
114/114 [==============================] 

Epoch 181/200
114/114 [==============================] - 0s 486us/step - loss: 2.2858 - acc: 0.1491 - val_loss: 2.3040 - val_acc: 0.1351
Epoch 182/200
114/114 [==============================] - 0s 497us/step - loss: 2.2858 - acc: 0.1491 - val_loss: 2.3040 - val_acc: 0.1351
Epoch 183/200
114/114 [==============================] - 0s 498us/step - loss: 2.2858 - acc: 0.1491 - val_loss: 2.3040 - val_acc: 0.1351
Epoch 184/200
114/114 [==============================] - 0s 473us/step - loss: 2.2862 - acc: 0.1491 - val_loss: 2.3040 - val_acc: 0.1351
Epoch 185/200
114/114 [==============================] - 0s 457us/step - loss: 2.2861 - acc: 0.1491 - val_loss: 2.3041 - val_acc: 0.1351
Epoch 186/200
114/114 [==============================] - 0s 483us/step - loss: 2.2843 - acc: 0.1491 - val_loss: 2.3041 - val_acc: 0.1351
Epoch 187/200
114/114 [==============================] - 0s 479us/step - loss: 2.2827 - acc: 0.1491 - val_loss: 2.3041 - val_acc: 0.1351
Epoch 188/200
114/114 [==================

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [29]:

from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               6422784   
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 257       
Total params: 21,137,729
Trainable params: 21,137,729
Non-trainable params: 0
_________________________________________________________________


In [8]:
from numba import cuda
cuda.select_device(1)
cuda.close()

IndexError: list index out of range